In [41]:
import pandas as pd
from joblib import dump, load
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [42]:
df = pd.read_csv('../../data/processed/spotify_songs_processed.csv', index_col=0)
df.dropna(axis=0, inplace=True)

In [43]:
df.describe()

,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
count,32828.000000,32828.000000,32828.000000,32828.000000,32828.000000,32828.000000,32828.000000,32828.000000,32828.000000,32828.000000,32828.000000,32828.000000,32828.000000
mean,42.483551,0.654850,0.698603,5.373949,-6.719529,0.565737,0.107053,0.175352,0.084760,0.190175,0.510556,120.883642,225796.829779
std,24.980476,0.145092,0.180916,3.611572,2.988641,0.495667,0.101307,0.219644,0.224245,0.154313,0.233152,26.903632,59836.492346
min,0.000000,0.000000,0.000175,0.000000,-46.448000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4000.000000
25%,24.000000,0.563000,0.581000,2.000000,-8.171250,0.000000,0.041000,0.015100,0.000000,0.092700,0.331000,99.961000,187804.500000
50%,45.000000,0.672000,0.721000,6.000000,-6.166000,1.000000,0.062500,0.080400,0.000016,0.127000,0.512000,121.984000,216000.000000
75%,62.000000,0.761000,0.840000,9.000000,-4.645000,1.000000,0.132000,0.255000,0.004830,0.248000,0.693000,133.918250,253581.250000
max,100.000000,0.983000,1.000000,11.000000,1.275000,1.000000,0.918000,0.994000,0.994000,0.996000,0.991000,239.440000,517810.000000


In [44]:
y = df['track_popularity']
genre_dummy = pd.get_dummies(df.playlist_genre)
X = pd.concat([genre_dummy, df.loc[:, 'danceability':'duration_ms']], axis=1)

In [45]:
# split the dataset into training and test
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [46]:
# standardise loudness, tempo and duration_ms
X_train['loudness'] = StandardScaler().fit(X_train.loc[:, ['loudness']]).transform(X_train.loc[:, ['loudness']])
X_train['tempo'] = StandardScaler().fit(X_train.loc[:, ['tempo']]).transform(X_train.loc[:, ['tempo']])
X_train['duration_ms'] = StandardScaler().fit(X_train.loc[:, ['duration_ms']]).transform(X_train.loc[:, ['duration_ms']])

In [47]:
# Grid search to fint the optiomal model
grid_param = {
    'n_estimators': [200, 300, 400, 500],
    'max_features': [2, 3, 4, 5],
    'max_depth': [3, 4, 5, 6]
}

model = RandomForestRegressor()

model_grid = GridSearchCV(model, param_grid=grid_param, cv=5)
model_grid.fit(X_train, y_train)

best_model = model_grid.best_estimator_
print(best_model)

RandomForestRegressor(max_depth=6, max_features=5, n_estimators=400)


In [48]:
y_train_pred = best_model.predict(X_train)
mean_squared_error(y_train, y_train_pred, squared=False)

23.196956237726656

In [49]:
# standardize the test data
X_test['loudness'] = StandardScaler().fit(X_test.loc[:, ['loudness']]).transform(X_test.loc[:, ['loudness']])
X_test['tempo'] = StandardScaler().fit(X_test.loc[:, ['tempo']]).transform(X_test.loc[:, ['tempo']])
X_test['duration_ms'] = StandardScaler().fit(X_test.loc[:, ['duration_ms']]).transform(X_test.loc[:, ['duration_ms']])

In [50]:
mean_squared_error(y_test, best_model.predict(X_test), squared=False)

23.804148860974852

In [51]:
# build the model using the whole dataset
X['loudness'] = StandardScaler().fit(X.loc[:, ['loudness']]).transform(X.loc[:, ['loudness']])
X['tempo'] = StandardScaler().fit(X.loc[:, ['tempo']]).transform(X.loc[:, ['tempo']])
X['duration_ms'] = StandardScaler().fit(X.loc[:, ['duration_ms']]).transform(X.loc[:, ['duration_ms']])

In [52]:
full_model = RandomForestRegressor(n_estimators=400, max_features=5, max_depth=7)
full_model.fit(X, y)

RandomForestRegressor(max_depth=7, max_features=5, n_estimators=400)

In [53]:
dump(full_model, 'spotify_model.joblib')

['spotify_model.joblib']

In [54]:
# save the scalar
loud_scalar = StandardScaler().fit(df.loc[:, ['loudness']])
dump(loud_scalar, 'loud_scalar.joblib')

['loud_scalar.joblib']

In [55]:
duration_scalar = StandardScaler().fit(df.loc[:, ['duration_ms']])
dump(duration_scalar, 'duration_scalar.joblib')

['duration_scalar.joblib']

In [56]:
tempo_scalar = StandardScaler().fit(df.loc[:, ['tempo']])
dump(tempo_scalar, 'tempo_scalar.joblib')

['tempo_scalar.joblib']

In [61]:
X.columns

Index(['edm', 'latin', 'pop', 'r&b', 'rap', 'rock', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms'],
      dtype='object')